In [8]:
import unittest
import torch
import numpy as np
from hw1 import train_DNN, Utility_function,simulate_optimal_path
import torch.nn as nn

In [9]:
# integration tests
class TestIntegration(unittest.TestCase):

    def setUp(self):
        """ initialize the model and parameters """
        self.T = 10
        self.a = 1
        self.p = 0.6
        self.A = 0.1
        self.B = -0.05
        self.r = 0.02
        self.W0 = np.random.uniform(1, 20)
        self.model = train_DNN(self.T, self.a, self.p, self.A, self.B, self.r,self.W0, num_epochs=100)

    def test_training_loss_decreasing(self):
        """ make sure the training loss is decreasing """
        W_values = np.arange(1, 20.5, 0.5)
        states, targets = [], []
        max_W, min_W = 20.0, 1.0
        # normalize the states
        normalize = lambda t, W: np.array([t / self.T, (W - min_W) / (max_W - min_W)])

        # generate the states and targets
        for t in range(self.T):
            for W in W_values:
                states.append(normalize(t, W))
                targets.append(Utility_function(W, self.a))  # calculate the target utility
        
        states = torch.FloatTensor(np.array(states)) # turn states into tensor
        targets = torch.FloatTensor(np.array(targets)).unsqueeze(1) # turn targets into tensor
        criterion = nn.MSELoss()

        losses = []
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001) # use Adam optimizer

        # train the model for 50 epochs
        for _ in range(50):  
            optimizer.zero_grad() # zero the gradients
            outputs = self.model(states) # forward pass
            loss = criterion(outputs, targets) # calculate the loss
            loss.backward() # backward pass
            optimizer.step() # update the weights
            losses.append(loss.item()) # record the loss

        # make sure the loss is decreasing
        self.assertTrue(losses[-1] < losses[0]) 

    def test_simulate_optimal_path(self):
        """ make sure the simulated optimal path is valid """
        actions, wealth = simulate_optimal_path(self.model, self.W0, self.T, self.p, self.A, self.B, self.r)

        # make sure the actions are within the valid range
        for x_t in actions:
            self.assertTrue(0.0 <= x_t <= 1.0)

        # make sure the wealth is not NaN or inf
        for W_t in wealth:
            self.assertFalse(np.isnan(W_t))
            self.assertFalse(np.isinf(W_t))

    def test_multiple_model_training(self):
        """ make sure the two models are trained independently """
        model_1 = train_DNN(self.T, self.a, self.p, self.A, self.B, self.r, self.W0, num_epochs=100)
        model_2 = train_DNN(self.T, self.a, self.p, self.A, self.B, self.r, self.W0, num_epochs=100)
        optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=0.001)
        optimizer_2 = torch.optim.Adam(model_2.parameters(), lr=0.001)

        # generate the states
        W_values = np.arange(1, 20.5, 0.5)
        states = [torch.FloatTensor([t / self.T, (W - 1) / (20 - 1)]) for t in range(self.T) for W in W_values]
        states = torch.stack(states)

        # train the two models
        optimizer_1.zero_grad()
        loss_1 = nn.MSELoss()(model_1(states), torch.zeros_like(model_1(states)))
        loss_1.backward()
        optimizer_1.step()

        optimizer_2.zero_grad()
        loss_2 = nn.MSELoss()(model_2(states), torch.zeros_like(model_2(states)))
        loss_2.backward()
        optimizer_2.step()

        params_1 = [p.clone().detach() for p in model_1.parameters()]
        params_2 = [p.clone().detach() for p in model_2.parameters()]

        for p1, p2 in zip(params_1, params_2):
            self.assertFalse(torch.equal(p1, p2))  # make sure the two models are trained independently

# run the integration tests
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)



Epoch 0, Loss: 0.0257
Epoch 0, Loss: 0.0082


.

Epoch 0, Loss: 0.0233
Epoch 0, Loss: 0.0885


.

Epoch 0, Loss: 0.0001


.
----------------------------------------------------------------------
Ran 3 tests in 6.130s

OK
